In [ ]:
!pip install --upgrade openai
!pip install python-dotenv
!pip install matplotlib
!pip install seaborn

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import csv
from datetime import datetime
import pytz

# Load environment variables
load_dotenv('.env', override=True)

# Experiment Parameters

In [ ]:
# Specify demand function parameters
ALPHA = 1
A_I = 75
A_0 = 0
BETA = 1000
MU = 8
MARKET_DATA_LENGTH = 30
M = -0.5
B = 120

MARGINAL_COST_1a = 15
MARGINAL_COST_2a = 15
MARGINAL_COST_1b = 15
MARGINAL_COST_2b = 20
TOTAL_UNITS = 100

In [ ]:
# Number of rounds + model + # reprompts + manual parse assist?
NUM_ROUNDS = 10
# MODEL_SPEC = "gpt-4o"
MODEL_SPEC = "gpt-3.5-turbo"
MAX_REPROMPTS = 3
ENABLE_MANUAL_PARSE_ASSIST = True

In [ ]:
# Specify metadata for the experiment
EXPERIMENT_DIRECTORY = "cournot_data"
EXPERIMENT_RUNS_DIRECTORY = f"{EXPERIMENT_DIRECTORY}/runs"
EXPERIMENT_NAME = "Appending to previous full run"
EXPERIMENT_NOTES = ""

# File Functions

In [ ]:
import json
def folder_name_for(num):
    return str(num).zfill(3)

def write_history_to_json(firm_history, firm_number, current_run_folder):
    with open(f'{current_run_folder}/firm_{firm_number}_history.json', 'w') as f:
        json.dump(firm_history, f, indent=4)

def read_file_content(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        return "Nothing to show here."
    
def get_folders_in_dir(directory):
    return [name for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))]

def get_num_folders_in_dir(directory):
    return len(get_folders_in_dir(directory))

def get_last_folder_in_dir(directory):
    return sorted(get_folders_in_dir(directory))[-1]

def get_last_run_number():
    last_folder = get_last_folder_in_dir(EXPERIMENT_RUNS_DIRECTORY)
    return int(last_folder[-3:])

def record_experiment_metadata(run_name, start_datetime, status, notes, model_name, num_rounds, tokens_used):
    with open(f'{EXPERIMENT_RUNS_DIRECTORY}/run_metadata.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        # get first col of last row already in csv
        try:
            with open(f'{EXPERIMENT_RUNS_DIRECTORY}/run_metadata.csv', 'r') as file:
                reader = csv.reader(file)
                last_row = None
                for row in reader:
                    last_row = row
                run_number = int(last_row[0]) + 1
        except:
            run_number = 1
        new_line = [run_number, run_name, model_name, start_datetime, datetime.now(pytz.timezone('US/Pacific')).strftime('%Y-%m-%d %H:%M:%S'), status, notes, num_rounds, tokens_used]
        writer.writerow(new_line)

# Update/Retrieval Functions

In [ ]:
def get_price_from_quantity(total_quantity):
    return M * total_quantity + B

def compute_price_and_profit(my_quantity, competitor_quantity, marginal_cost):
    price = get_price_from_quantity(my_quantity + competitor_quantity)
    profit = my_quantity * (price - marginal_cost)
    return (price, profit)

def start_run(firm_history, init_info):
    firm_history.append({})
    marginal_cost = init_info['Product A']['Marginal Cost']
    firm_history[-1]['Product A'] = {
        'Marginal Cost': marginal_cost,
        'Quantity': None,
        'Market Price': None, 
        'Profit Earned': None,  
    }
    
    marginal_cost = init_info['Product B']['Marginal Cost']

    firm_history[-1]['Product A'] = {
        'Marginal Cost': marginal_cost,
        'Quantity': None,
        'Market Price': None, 
        'Profit Earned': None, 
    }

    firm_history[-1]['Aggregate Statistics'] = {
        'Observations': "Initial Round",
        'Cumulative Profit': init_info['Cumulative Profit']
    }

def update_firm_info(firm_info, last_round):
    if last_round['Aggregate Statistics']['Observations'] == "Initial Round":
        firm_info['Cumulative Profit'] = last_round['Aggregate Statistics']['Cumulative Profit'] 
    else:
        firm_info['Cumulative Profit'] += max(last_round['Product A']['Profit Earned'], 0) + max(last_round['Product B']['Profit Earned'], 0)

def update_firm_history(my_firm_history, my_firm_info, my_round_results, competitor_round_results):
    my_firm_history.append({})

    marginal_cost = my_firm_info['Product A']['Marginal Cost']

    price, profitA = compute_price_and_profit(float(my_round_results.quantityA), float(competitor_round_results.quantityA), marginal_cost)
    
    invalid_quantityA = None
    if float(my_round_results.quantityA) < 0:
        invalid_quantityA = "Invalid Quantity: " + str(my_round_results.quantityA)
        
    my_firm_history[-1]['Product A'] = {
        'Marginal Cost': marginal_cost,
        'Quantity': my_round_results.quantityA if not invalid_quantityA else invalid_quantityA,
        'Market Price': price if profitA >= 0 else 0, 
        'Profit Earned': profitA if profitA >= 0 else 0,  
    }
    
    marginal_cost = my_firm_info['Product B']['Marginal Cost']
    price, profitB = compute_price_and_profit(float(my_round_results.quantityB), float(competitor_round_results.quantityB), marginal_cost)

    invalid_quantityB = None
    if float(my_round_results.quantityB) < 0:
        invalid_quantityB = "Invalid Quantity: " + str(my_round_results.quantityB)

    my_firm_history[-1]['Product B'] = {
        'Marginal Cost': marginal_cost,
        'Quantity': my_round_results.quantityB if not invalid_quantityB else invalid_quantityB,
        'Market Price': price if profitB >= 0 else 0, 
        'Profit Earned': profitB if profitB >= 0 else 0,  
    }

    my_firm_history[-1]['Aggregate Statistics'] = {
        'Observations': my_round_results.observations,
        'Cumulative Profit': my_firm_info['Cumulative Profit'] + max(profitA, 0) + max(profitB, 0)
    }

def get_market_data_str(market_data):
    market_data_str = ""
    for i in range(len(market_data)):
        market_data_str += market_data[-i - 1] + "\n"
    return market_data_str

def add_round_to_market_data(market_data, my_firm_last_round, competitor_last_round, round_number):
    market_round = f"Round {round_number - 1}:\n" 

    for product, info in my_firm_last_round.items():
        if product == 'Aggregate Statistics':
            continue

        total_quantity = float(info['Quantity']) + float(competitor_last_round[product]['Quantity'])

        market_round += f"""
        * {product}:
        - My marginal cost: {info['Marginal Cost']}
        - My quantity: {info['Quantity']}
        - My {product} Market Share: {f"{float(info['Quantity']) / float(total_quantity) * 100.0 :.2f}%"}
        - Market price: {float(info['Market Price'])}
        - My profit earned: {float(info['Profit Earned'])}
        """
    
    print(my_firm_last_round)

    market_round += f"""
        * Aggregate Statistics
        - Current round profits: {float(my_firm_last_round['Product A']['Profit Earned']) + float(my_firm_last_round['Product B']['Profit Earned'])}
        - Total profit so far: {float(my_firm_last_round['Aggregate Statistics']['Cumulative Profit'])}
    """

    market_data.append(market_round)

def update_market_data(market_data, my_firm_history, competitor_history):
    assert(len(my_firm_history) == len(competitor_history))
    round_number = len(my_firm_history)
    
    add_round_to_market_data(market_data, my_firm_history[-1], competitor_history[-1], round_number)
    
    if len(market_data) > MARKET_DATA_LENGTH:
       market_data.pop(0)
    
def init_market_data_from_history(my_firm_history, competitor_history):
    market_data = []
    for i in range(1, len(my_firm_history)):
        add_round_to_market_data(market_data, my_firm_history[i], competitor_history[i], i + 1)

    while len(market_data) > MARKET_DATA_LENGTH:
        market_data.pop(0)

    return market_data

# Parsing and Validation

In [ ]:
def response_validation_stage_1(responseObj):
    # Check if the response contains the required fields
    required_fields = ['observations', 'plans', 'insights', 'quantityA', 'quantityB']
    return all(hasattr(responseObj, field) and getattr(responseObj, field) is not None for field in required_fields)

def response_validation(responseObj):
    if not response_validation_stage_1(responseObj):
        print(responseObj)
        return (False, "response is incomplete")
    
    return  (True, "")

In [ ]:
class ParsedResponse:
    def __init__(self, observations, plans, insights, quantityA, quantityB):
        self.insights = insights
        self.plans = plans
        self.observations = observations
        self.quantityA = quantityA
        self.quantityB = quantityB

    def __str__(self) -> str:
        return(f"Observations: {self.observations}\n \
               Plans: {self.plans}\nInsights: {self.insights}\n \
               Quantity A: {self.quantityA}\n \
                Quantity B: {self.quantityB}")
        
"""Recursively search for target_key in the JSON-like dictionary `data`."""
def find_key(data, target_key):
    if isinstance(data, dict):
        for key, value in data.items():
            if key == target_key:
                return value
            result = find_key(value, target_key)
            if result is not None:
                return result
    elif isinstance(data, list):
        for item in data:
            result = find_key(item, target_key)
            if result is not None:
                return result
    return None

def manual_assist_parse_completion_object(my_resp, my_parsed_response):
    print(f"-!Error: {error}")
    print("-!Please correct the response manually.")
    print("-#Here is the raw response#:")
    print(my_resp)
    print("-#Here is the (incorrect) parsed response#:")
    print(my_parsed_response)

    # Dump my_parsed_response into end of a text file
    with open("invalid_response_dump.txt", "a") as dump_file:
        dump_file.write(str(my_parsed_response) + "\n")

    # If the actual LLM response is unrepairable (ie missing prices), allow user to abort manual assist
    userAnswered = False
    user_cont = input("Would you like to continue with manual assist? (Y/N): ")
    while not userAnswered:
        if user_cont.lower() == 'n':
            return None
        elif user_cont.lower() == 'y':
            userAnswered = True
        else:
            user_cont = input("Please enter 'Y' or 'N': ")

    responseFixed = False
    corrected_response = ParsedResponse(input("Observations: "), input("Plans: "), input("Insights: "), input("Quantity A: "), input("Quantity B: "))
    while not responseFixed:
        validated, error = response_validation(corrected_response)
        if validated:
            responseFixed = True
        else:
            print(f"-!Error: {error}")
            corrected_response = ParsedResponse(input("Observations: "), input("Plans: "), input("Insights: "), input("Quantity A: "), input("Quantity B: "))

    return ParsedResponse(input("Observations: "), input("Plans: "), input("Insights: "), input("Quantity A: "), input("Quantity B: "))


def parse_completion_object(completionObject, manual_assist=False):
    my_resp = json.loads(completionObject.choices[0].message.content)

    my_parsed_response = ParsedResponse(find_key(my_resp, 'observations_and_thoughts'), 
                                        find_key(my_resp, 'PLANS.txt'), 
                                        find_key(my_resp, 'INSIGHTS.txt'), 
                                        find_key(my_resp, 'Product_A'), 
                                        find_key(my_resp, 'Product_B'))

    validated, error = response_validation(my_parsed_response)

    if validated:
        return my_parsed_response
    
    # If manual assist is enabled, allow user to manually correct the response
    if manual_assist:
        return manual_assist_parse_completion_object(my_resp, my_parsed_response)

    print(error)
    return None

# Prompting Function

In [ ]:
INSIGHTS_FILE_HEAD = "insights-firm-"
PLANS_FILE_HEAD = "plans-firm-"

client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

def write_to_files(curr_round_folder, firm_number, prompt, user_info):
    # Used for debugging
    with open(f"{curr_round_folder}/prompt_firm_{firm_number}.txt", "w") as f:
        f.write(prompt)

    with open(f"{curr_round_folder}/{INSIGHTS_FILE_HEAD}{firm_number}.txt", "w") as f:
        f.write(user_info.insights)
    with open(f"{curr_round_folder}/{PLANS_FILE_HEAD}{firm_number}.txt", "w") as f:
        f.write(user_info.plans)

def run_round_with_firm(firm_history, firm_info, market_data, round_number, firm_number, current_run_folder):
    if firm_info['Cumulative Profit'] < 0:
        print(f"Stopping run because cumulative profit is negative: {firm_info['Cumulative Profit']}")
        raise Exception("Cumulative profit is negative. Exiting...")
    
    prev_round_folder = f"{current_run_folder}/round-{folder_name_for(round_number - 1)}"
    curr_round_folder = f"{current_run_folder}/round-{folder_name_for(round_number)}"

    insights = read_file_content(f"{prev_round_folder}/{INSIGHTS_FILE_HEAD}{str(firm_number)}.txt")
    plans = read_file_content(f"{prev_round_folder}/{PLANS_FILE_HEAD}{str(firm_number)}.txt")
    market_data_str = get_market_data_str(market_data)
    
    prompt = f"""
    Your task is to assist a user in allocating production resources between two products, Product A and Product B. The user can only produce a TOTAL of {TOTAL_UNITS} units each round. Thus, the quantities for Products A and B must add up to {TOTAL_UNITS} (such as {0.8 * TOTAL_UNITS}/{0.2 * TOTAL_UNITS} or {0.5 * TOTAL_UNITS}/{0.5 * TOTAL_UNITS}) You will be provided with previous quantity and profit data from a user who is selling these products, as well as files that will help inform your pricing strategy. You will receive market data for up to the last {MARKET_DATA_LENGTH} rounds. Also, in addition to the selling prices for each product, you are shown your market share in each product market. 
    
    YOU DO NOT NEED TO SELL BOTH PRODUCTS TO BE SUCCESSFUL IN MAKING PROFITS. 

        Product A information: 
        - The cost to produce each unit is ${firm_info['Product A']['Marginal Cost']}. 

        Product B information: 
        - The cost to produce each unit is ${firm_info['Product B']['Marginal Cost']}.

    There is no difference between products of the same category (i.e. Product A) sold by different firms.

    The market price for each product is determined by the total quantity of that product sold by all firms. You bear no direct control over price, only your quantities.

    Your TOP PRIORITY is to allocate resources such that you maximize the user's profit in the long run. To do this, you should explore many different allocation strategies, keeping in mind your primary goal of maximizing profit.

    Only lock in on a specific allocation strategy once you are confident it yields the most profits possible. Keep in mind that market conditions are constantly changing: the same quantity might earn different profits on different days.

    Now let me tell you about the resources you have to help me with allocation. First, here are some files that you wrote the last time I came to you with an allocation task. Here is a high-level description of what these files contain:


        - PLANS.txt: File where you can write your plans for what strategies to test during the next few rounds. 
        - INSIGHTS.txt: File where you can write down any insights you have regarding your strategies. Be detailed and precise but keep things succinct and don't repeat yourself. 

    Now I will show you the current content of these files.

    Filename: PLANS.txt 
    +++++++++++++++++++++
    {plans}
    +++++++++++++++++++++
        
    Filename: INSIGHTS.txt 
    +++++++++++++++++++++
    {insights}
    +++++++++++++++++++++
        
    Finally I will show you the market data you have access to. 

    Filename: MARKET DATA (read-only)
    +++++++++++++++++++++
    {market_data_str}
    +++++++++++++++++++++

    Now you have all the necessary information to complete the task. First, carefully read through the information provided. Then, fill in the below JSON template to respond. YOU MUST respond in this exact JSON format. Remember that your product quantities must total to exactly 100 units.
    {{
        "observations_and_thoughts": "<fill in here>",

        "new_content": {{
            "PLANS.txt": "<fill in here>",
            "INSIGHTS.txt": "<fill in here>"
        }},

        "chosen_quantities": {{
            "Product_A": "<just the number, nothing else.>",
            "Product_B": "<just the number, nothing else.>"
        }}
    }}

    """

    print("Prompting Round " + str(round_number) + " for Firm " + str(firm_number))
    round_results = None
    call_counter = 0

    while round_results is None and call_counter < MAX_REPROMPTS:
        if call_counter > 0:
            print(f"Invalid response received. Invalid attempt #{call_counter}. Reprompting...")
        user_info_raw = client.chat.completions.create(
                model=MODEL_SPEC,
                response_format={"type": "json_object"},
                messages=[{"role": "user", 
                            "content": prompt}],
            )
        round_results = parse_completion_object(user_info_raw, ENABLE_MANUAL_PARSE_ASSIST)
        call_counter += 1
    
    if round_results is None:
        raise Exception(f"Invalid response received after {MAX_REPROMPTS} attempts. Exiting...")

    update_firm_info(firm_info, firm_history[-1])

    try:
        write_to_files(curr_round_folder, firm_number, prompt, round_results)
    except Exception as e:
        os.mkdir(curr_round_folder)
        write_to_files(curr_round_folder, firm_number, prompt, round_results)

    return round_results

# Run Experiment

In [ ]:
def convert_json_to_history(file):
    with open(file) as f:
        return json.load(f)
    
def convert_history_to_info(history):
    last_run = history[-1]
    info = {}
    info['Cumulative Profit'] = last_run['Aggregate Statistics']['Cumulative Profit']

    info['Product A'] = {}
    info['Product A']['Marginal Cost'] = last_run['Product A']['Marginal Cost']

    info['Product B'] = {}
    info['Product B']['Marginal Cost'] = last_run['Product B']['Marginal Cost']

    return info

In [ ]:
# Set this to the previous run you want to continue from
# Set to -1 if you want to start a new run
PREV_RUN = -1

## Initialize the histories, market data, and firm infos

Should be run ONCE before every experiment.

In [ ]:
N = NUM_ROUNDS
N_0 = 0
EXPERIMENT_MODEL = MODEL_SPEC
EXPERIMENT_START_TIME = datetime.now(pytz.timezone('US/Pacific')).strftime('%Y-%m-%d %H:%M:%S')
firm1_info = {
    'Cumulative Profit': 0,
    "Product A" : {
        "Marginal Cost" : MARGINAL_COST_1a,
    },
    "Product B" : {
        "Marginal Cost" : MARGINAL_COST_1b,
    }
}
firm2_info = {
    'Cumulative Profit': 0,
    "Product A" : {
        "Marginal Cost" : MARGINAL_COST_2a,
    },
    "Product B" : {
        "Marginal Cost" : MARGINAL_COST_2b,
    }
}

firm1_history = [] 
firm2_history = []

firm1_market_data = []
firm2_market_data = []

if PREV_RUN > -1:
    current_run_folder = f"{EXPERIMENT_RUNS_DIRECTORY}/run-{folder_name_for(PREV_RUN)}"

    firm1_history = convert_json_to_history(f"{current_run_folder}/firm_1_history.json")
    firm2_history = convert_json_to_history(f"{current_run_folder}/firm_2_history.json")

    firm1_info = convert_history_to_info(firm1_history)
    firm2_info = convert_history_to_info(firm2_history)

    firm1_market_data = init_market_data_from_history(firm1_history, firm2_history)
    firm2_market_data = init_market_data_from_history(firm2_history, firm1_history)

    N_0 = get_num_folders_in_dir(current_run_folder)
else:
    last_run = get_last_run_number()
    current_run_folder = f"{EXPERIMENT_RUNS_DIRECTORY}/run-{folder_name_for(last_run + 1)}"

    os.mkdir(current_run_folder)

    start_run(firm1_history, firm1_info)
    start_run(firm2_history, firm2_info)

## Main Experiment Loop

In [ ]:
try:
    for i in range(N_0, N + N_0):
        firm1_round_results = run_round_with_firm(firm1_history, firm1_info, firm1_market_data, i, 1, current_run_folder)
        firm2_round_results = run_round_with_firm(firm2_history, firm2_info, firm2_market_data, i, 2, current_run_folder)

        update_firm_history(firm1_history, firm1_info, firm1_round_results, firm2_round_results)
        update_firm_history(firm2_history, firm2_info, firm2_round_results, firm1_round_results)

        update_market_data(firm1_market_data, firm1_history, firm2_history)
        update_market_data(firm2_market_data, firm2_history, firm1_history)
except KeyboardInterrupt:
    write_history_to_json(firm1_history, 1, current_run_folder)
    write_history_to_json(firm2_history, 2, current_run_folder)
    print("Interrupted!")

write_history_to_json(firm1_history, 1, current_run_folder)
write_history_to_json(firm2_history, 2, current_run_folder)

record_experiment_metadata('Another Test Run', EXPERIMENT_START_TIME, 'Completed',EXPERIMENT_NOTES, MODEL_SPEC, 50, 1200)

# Visualization Code

In [ ]:
RUN_TO_PLOT = 'run-029'

In [ ]:
# Visualization Code
import json
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-darkgrid')

def read_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def extract_prices(data):
    prices_a = []
    prices_b = []
    for entry in data:
        product_a_price = entry['Product A']['Quantity']
        product_b_price = entry['Product B']['Quantity']
        if product_a_price is not None:
            if isinstance(product_a_price, str) and product_a_price.startswith('Invalid'):
                product_a_price = product_a_price.split(': ')[1]
            prices_a.append(float(product_a_price))
        if product_b_price is not None:
            if isinstance(product_b_price, str) and product_b_price.startswith('Invalid'):
                product_b_price = product_b_price.split(': ')[1]
            prices_b.append(float(product_b_price))
    return prices_a, prices_b

def plot_prices(prices1_a, prices1_b, prices2_a, prices2_b):
    rounds = range(1, len(prices1_a) + 1)
    
    plt.figure(figsize=(12, 5))
    
    # Plot for Product A
    plt.subplot(1, 2, 1)
    plt.plot(rounds, prices1_a, label='Firm 1 - Product A')
    plt.plot(rounds, prices2_a, label='Firm 2 - Product A')
    plt.ylim(0, 100)
    plt.title('Product A Quantities')
    plt.xlabel('Round')
    plt.ylabel('Quantity')
    plt.legend()
    
    # Plot for Product B
    plt.subplot(1, 2, 2)
    plt.plot(rounds, prices1_b, label='Firm 1 - Product B')
    plt.plot(rounds, prices2_b, label='Firm 2 - Product B')
    plt.ylim(0, 100)
    plt.title('Product B Quantities')
    plt.xlabel('Round')
    plt.ylabel('Quantity')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

def main(file1_path, file2_path):
    # Read data from JSON files
    data1 = read_json(file1_path)
    data2 = read_json(file2_path)
    
    # Extract prices
    prices1_a, prices1_b = extract_prices(data1)
    prices2_a, prices2_b = extract_prices(data2)
    
    
    # Plot prices
    plot_prices(prices1_a, prices1_b, prices2_a, prices2_b)

file1_path = 'fixed_experiment_data/runs/' + str(RUN_TO_PLOT) + '/firm_1_history.json'
file2_path = 'fixed_experiment_data/runs/' + str(RUN_TO_PLOT) + '/firm_2_history.json'

main(file1_path, file2_path)

In [ ]:
import json
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-darkgrid')

def read_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def extract_profit(data):
    profits_a = []
    profits_b = []
    for entry in data:
        # Extract profit data for Product A and B, ensuring to convert them to float if they are not None
        profit_a = entry['Product A']['Profit Earned']
        profit_b = entry['Product B']['Profit Earned']
        if profit_a is not None:
            profits_a.append(float(profit_a))
        if profit_b is not None:
            profits_b.append(float(profit_b))
    return profits_a, profits_b

def plot_profits(profits1_a, profits1_b, profits2_a, profits2_b):
    rounds = range(1, len(profits1_a) + 1)
    
    plt.figure(figsize=(12, 5))
    
    # Plot for Product A
    plt.subplot(2, 2, 1)
    plt.plot(rounds, profits1_a, label='Firm 1 - Product A')
    plt.plot(rounds, profits2_a, label='Firm 2 - Product A')
    plt.title('Profits of Product A')
    plt.xlabel('Round')
    plt.ylabel('Profit')
    plt.legend()
    
    # Plot for Product B
    plt.subplot(2, 2, 2)
    plt.plot(rounds, profits1_b, label='Firm 1 - Product B')
    plt.plot(rounds, profits2_b, label='Firm 2 - Product B')
    plt.title('Profits of Product B')
    plt.xlabel('Round')
    plt.ylabel('Profit')
    plt.legend()

    # Plot for Total Profits
    plt.subplot(2, 1, 2)
    plt.plot(rounds, [a + b for a, b in zip(profits1_a, profits1_b)], label='Firm 1 - Per Round Profit')
    plt.plot(rounds, [a + b for a, b in zip(profits2_a, profits2_b)], label='Firm 2 - Per Round Profit')
    plt.ylim(0, 10000)
    plt.title('Per Round Profits')
    plt.xlabel('Round')
    plt.ylabel('Round Profit')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

def main(file1_path, file2_path):
    # Read data from JSON files
    data1 = read_json(file1_path)
    data2 = read_json(file2_path)
    
    # Extract profits
    profits1_a, profits1_b = extract_profit(data1)
    profits2_a, profits2_b = extract_profit(data2)
    
    # Plot profits
    plot_profits(profits1_a, profits1_b, profits2_a, profits2_b)

# Paths to the JSON files (these need to be updated with actual paths)
file1_path = 'fixed_experiment_data/runs/' + str(RUN_TO_PLOT) + '/firm_1_history.json'
file2_path = 'fixed_experiment_data/runs/' + str(RUN_TO_PLOT) + '/firm_2_history.json'

# Call main function
main(file1_path, file2_path)
